# Grad-CAM Implementation for Retinal Image Dataset

This notebook demonstrates how to use Grad-CAM (Gradient-weighted Class Activation Mapping) to visualize important regions of an image that contribute to a deep learning model's prediction. The model used is a CNN (Convolutional Neural Network) trained to classify retinal images into two classes: **Non-CVD** (label 0) and **CVD** (label 1).

### Code Implementation

The following code performs the following steps:

1. **Loading the model**: The pre-trained CNN model is loaded from the file `model.h5`.
2. **Compiling the model**: The model is compiled with a dummy optimizer and loss function to ensure layers are initialized properly.
3. **Loading the dataset**: The dataset consists of retinal images stored in folders `0` and `1`, representing the classes **Non-CVD** and **CVD**, respectively.
4. **Selecting random images**: Three random images are selected from the dataset for visualization.
5. **Computing Grad-CAM**: Grad-CAM is applied to the selected images, and the heatmaps are visualized alongside the original images.

In [6]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

# Load the model
model = load_model('model.h5')

# Compile the model to initialize the layers properly
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Function to load and preprocess dataset
def load_data(dataset_path, img_size=(224, 224)):
    data, labels = [], []
    for label in [0, 1]:  # 0: Non-CVD, 1: CVD
        folder_path = os.path.join(dataset_path, str(label))
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img, img_size)
                data.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    return np.array(data), np.array(labels)

# Select 3 random images for Grad-CAM
data, labels = load_data('data')  # Path to the 'data' folder containing '0' and '1'
indices = np.random.choice(len(data), 3, replace=False)
selected_images = data[indices]
selected_labels = labels[indices]

# Function to compute Grad-CAM
def compute_gradcam(model, image, label, layer_name='conv2d_2'):
    # Prepare the image for prediction
    image_input = np.expand_dims(image, axis=0)  # Add batch dimension
    image_input = image_input / 255.0  # Normalize

    # Get the model's last convolutional layer and its output
    last_conv_layer = model.get_layer(layer_name)
    heatmap_model = tf.keras.models.Model(
        inputs=[model.inputs],
        outputs=[last_conv_layer.output, model.output]
    )
    
    with tf.GradientTape() as tape:
        conv_output, predictions = heatmap_model(image_input)
        loss = predictions[:, label]
    
    # Compute the gradients of the loss with respect to the last convolutional layer
    grads = tape.gradient(loss, conv_output)
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    
    # Multiply the gradients with the convolutional output
    conv_output = conv_output[0]
    pooled_grads = pooled_grads.numpy()
    conv_output = conv_output.numpy()
    
    for i in range(conv_output.shape[-1]):
        conv_output[:, :, i] *= pooled_grads[i]
    
    # Generate the heatmap
    heatmap = np.mean(conv_output, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    # Resize heatmap to match the image size
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    return heatmap

# Visualize the Grad-CAM for the selected images
for i, image in enumerate(selected_images):
    label = selected_labels[i]
    
    # Compute Grad-CAM for this image
    heatmap = compute_gradcam(model, image, label)
    
    # Plot the original image with the Grad-CAM overlay
    plt.figure(figsize=(10, 8))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f'Original Image (Label: {label})')
    
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.imshow(heatmap, cmap='jet', alpha=0.5)  # Overlay heatmap with 50% transparency
    plt.title(f'Grad-CAM (Label: {label})')
    plt.show()


AttributeError: The layer sequential has never been called and thus has no defined output.

### Note on Grad-CAM Error

<span style="color:green">I faced a challenge while implementing Grad-CAM for the first time. Unfortunately, due to time constraints, I was unable to resolve the error fully this time. However, I tried my best to follow the steps, and I believe I’ve captured the key aspects of the process. I will revisit this in the future and ensure the implementation is smooth and works perfectly.</span>
